In [ ]:
import pandas as pd
import os
from glob import glob
from reports import get_average_classification_report
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
no_pt_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=None_target=*/classification_report.csv'
pt_st_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=ST_target=*/classification_report.csv'
pt_t_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=T_target=*/classification_report.csv'
kfold_num = 1

In [ ]:
def show_result(filepaths_format, kfold_num=1):
    kfold_reports = []
    for i in range(0, kfold_num):
        filepaths = glob(filepaths_format.format(i))

        records = []
        for filepath in filepaths:
            source, post, target = os.path.basename(os.path.dirname(filepath)).split('_')
            source, target = source.replace('source=',''), target.replace('target=', '')

            df = pd.read_csv(filepath)
            df.set_index('Unnamed: 0', inplace=True)
            acc = 100*df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]

            records.append(('{}->{}'.format(source, target), float(acc)))

        result_df = pd.DataFrame(records, columns = ['source->target', '{}_accuracy'])
        kfold_reports.append(result_df)

    result_df = get_average_classification_report(kfold_reports)
    result_df.loc['Average'] = result_df.mean()
    return result_df

* Post-training baselines
    1. No post-training
    2. Source+Target MLM
    3. Target MLM
* 파인튜닝: Raw 소스 (train), Raw 타겟 (test)
* No k-fold

In [ ]:
no_pt = show_result(no_pt_filepaths_format, kfold_num=kfold_num)
no_pt.columns = ['No post-training']

pt_st = show_result(pt_st_filepaths_format, kfold_num=kfold_num)
pt_st.columns = ['Source+Target MLM']

pt_t = show_result(pt_t_filepaths_format, kfold_num=kfold_num)
pt_t.columns = ['Target MLM']

result = pd.concat([no_pt, pt_st, pt_t], axis=1)
result